In [0]:
#read the data
df1 = spark.read.format("csv").option("delimiter", "\t").option("header","true").option("inferschema","true").load("dbfs:/FileStore/shared_uploads/Satadru1998@gmail.com/Data_Analysis_Play.tsv")

In [0]:
#Print first 5 records
df1.show(5) 

+--------+--------+--------------------+--------------------+----------+
order_id|quantity| item_name| choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
 1| 1|Chips and Fresh T...| NULL| $2.39 |
 1| 1| Izze| [Clementine]| $3.39 |
 1| 1| Nantucket Nectar| [Apple]| $3.39 |
 1| 1|Chips and Tomatil...| NULL| $2.39 |
 2| 2| Chicken Bowl|[Tomatillo-Red Ch...| $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows

In [0]:
#Print total records
df1.count(),len(df1.columns)

Out[29]: (4622, 5)

In [0]:
#Print the name of all the columns
df1.columns

Out[30]: ['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [0]:
#Print type of variables
df1.printSchema()

root
-- order_id: integer (nullable = true)
-- quantity: integer (nullable = true)
-- item_name: string (nullable = true)
-- choice_description: string (nullable = true)
-- item_price: string (nullable = true)

In [0]:
#Print the last 7 records

In [0]:
from pyspark.sql.functions import *

In [0]:
df1=df1.withColumn("Row_id",monotonically_increasing_id());
df1.show(5)

+--------+--------+--------------------+--------------------+----------+------+
order_id|quantity| item_name| choice_description|item_price|Row_id|
+--------+--------+--------------------+--------------------+----------+------+
 1| 1|Chips and Fresh T...| NULL| $2.39 | 0|
 1| 1| Izze| [Clementine]| $3.39 | 1|
 1| 1| Nantucket Nectar| [Apple]| $3.39 | 2|
 1| 1|Chips and Tomatil...| NULL| $2.39 | 3|
 2| 2| Chicken Bowl|[Tomatillo-Red Ch...| $16.98 | 4|
+--------+--------+--------------------+--------------------+----------+------+
only showing top 5 rows

In [0]:
tail_part=df1.filter(df1['Row_id']>=df1.count()-7);
tail_part.show()

+--------+--------+-------------------+--------------------+----------+------+
order_id|quantity| item_name| choice_description|item_price|Row_id|
+--------+--------+-------------------+--------------------+----------+------+
 1832| 1| Chicken Soft Tacos|[Fresh Tomato Sal...| $8.75 | 4615|
 1832| 1|Chips and Guacamole| NULL| $4.45 | 4616|
 1833| 1| Steak Burrito|[Fresh Tomato Sal...| $11.75 | 4617|
 1833| 1| Steak Burrito|[Fresh Tomato Sal...| $11.75 | 4618|
 1834| 1| Chicken Salad Bowl|[Fresh Tomato Sal...| $11.25 | 4619|
 1834| 1| Chicken Salad Bowl|[Fresh Tomato Sal...| $8.75 | 4620|
 1834| 1| Chicken Salad Bowl|[Fresh Tomato Sal...| $8.75 | 4621|
+--------+--------+-------------------+--------------------+----------+------+

In [0]:
#What is the most ordered item? & how many items were ordered?
df1.groupBy('item_name').agg(sum('quantity').alias("Total_number_of_orders")).orderBy("Total_number_of_orders",ascending=False).show(1)

+------------+----------------------+
 item_name|Total_number_of_orders|
+------------+----------------------+
Chicken Bowl| 761|
+------------+----------------------+
only showing top 1 row

In [0]:
#What is the most ordered item in the choice_description column & how many items were ordered?
df2=df1.filter(df1['choice_description']!='NULL');
df2.groupBy('choice_description').agg(sum('quantity').alias("Total_number_of_orders")).orderBy("Total_number_of_orders",ascending=False).show(1)

+------------------+----------------------+
choice_description|Total_number_of_orders|
+------------------+----------------------+
 [Diet Coke]| 159|
+------------------+----------------------+
only showing top 1 row

In [0]:
#Turn the item price into float
def convertera(x):
  ms=float(x[1:]);
  return ms;

In [0]:
from pyspark.sql.types import *

In [0]:
uff_for_convertera=udf(convertera,FloatType());

In [0]:
df1=df1.withColumn('item_price',uff_for_convertera(df1['item_price']));
df1.show()

+--------+--------+--------------------+--------------------+----------+------+
order_id|quantity| item_name| choice_description|item_price|Row_id|
+--------+--------+--------------------+--------------------+----------+------+
 1| 1|Chips and Fresh T...| NULL| 2.39| 0|
 1| 1| Izze| [Clementine]| 3.39| 1|
 1| 1| Nantucket Nectar| [Apple]| 3.39| 2|
 1| 1|Chips and Tomatil...| NULL| 2.39| 3|
 2| 2| Chicken Bowl|[Tomatillo-Red Ch...| 16.98| 4|
 3| 1| Chicken Bowl|[Fresh Tomato Sal...| 10.98| 5|
 3| 1| Side of Chips| NULL| 1.69| 6|
 4| 1| Steak Burrito|[Tomatillo Red Ch...| 11.75| 7|
 4| 1| Steak Soft Tacos|[Tomatillo Green ...| 9.25| 8|
 5| 1| Steak Burrito|[Fresh Tomato Sal...| 9.25| 9|
 5| 1| Chips and Guacamole| NULL| 4.45| 10|
 6| 1|Chicken Crispy Tacos|[Roasted Chili Co...| 8.75| 11|
 6| 1| Chicken Soft Tacos|[Roasted Chili Co...| 8.75| 12|
 7| 1| Chicken Bowl|[Fresh Tomato Sal...| 11.25| 13|
 7| 1| Chips and Guacamole| NULL| 4.45| 14|
 8| 1|Chips and Tomatil...| NULL| 2.39| 15|
 8| 1| Chicken Burrito|[Tomatillo-Green ...| 8.49| 16|
 9| 1| Chicken Burrito|[Fresh Tomato Sal...| 8.49| 17|
 9| 2| Canned Soda| [Sprite]| 2.18| 18|
 10| 1| Chicken Bowl|[Tomatillo Red Ch...| 8.75| 19|
+--------+--------+--------------------+--------------------+----------+------+
only showing top 20 rows

In [0]:
#How much was the revenue for the period in the dataset?

In [0]:
df1.withColumn('Revenue',df1['quantity']*df1['item_price']).select(sum('Revenue')).show()

+----------------+
 sum(Revenue)|
+----------------+
39237.0197327137|
+----------------+

In [0]:
#print a data frame with only two columns item_name and item_price

In [0]:
df1.select(['item_name','item_price']).show()

+--------------------+----------+
 item_name|item_price|
+--------------------+----------+
Chips and Fresh T...| 2.39|
 Izze| 3.39|
 Nantucket Nectar| 3.39|
Chips and Tomatil...| 2.39|
 Chicken Bowl| 16.98|
 Chicken Bowl| 10.98|
 Side of Chips| 1.69|
 Steak Burrito| 11.75|
 Steak Soft Tacos| 9.25|
 Steak Burrito| 9.25|
 Chips and Guacamole| 4.45|
Chicken Crispy Tacos| 8.75|
 Chicken Soft Tacos| 8.75|
 Chicken Bowl| 11.25|
 Chips and Guacamole| 4.45|
Chips and Tomatil...| 2.39|
 Chicken Burrito| 8.49|
 Chicken Burrito| 8.49|
 Canned Soda| 2.18|
 Chicken Bowl| 8.75|
+--------------------+----------+
only showing top 20 rows

In [0]:
# delete the duplicates in item_name and quantity
filtered_data=df1.drop_duplicates(['item_name','quantity']);
# select only the products with quantity equals to 1
prod_1=filtered_data.filter(filtered_data['quantity']==1);
# select only the item_name and item_price columns
proper_selection=prod_1.select('item_name','item_price')
# sort the values from the most to less expensive
proper_selection.orderBy('item_price',ascending=False).show(proper_selection.count())

+--------------------+----------+
 item_name|item_price|
+--------------------+----------+
 Barbacoa Salad Bowl| 11.89|
 Steak Salad Bowl| 11.89|
 Carnitas Salad Bowl| 11.89|
 Barbacoa Bowl| 11.75|
Barbacoa Crispy T...| 11.75|
 Steak Burrito| 11.75|
 Veggie Salad Bowl| 11.25|
 Veggie Bowl| 11.25|
 Veggie Burrito| 11.25|
 Veggie Soft Tacos| 11.25|
 Chicken Salad| 10.98|
 Chicken Bowl| 10.98|
Carnitas Crispy T...| 9.25|
 Barbacoa Soft Tacos| 9.25|
 Carnitas Soft Tacos| 9.25|
 Steak Soft Tacos| 9.25|
 Steak Crispy Tacos| 9.25|
 Carnitas Salad| 8.99|
 Carnitas Bowl| 8.99|
 Carnitas Burrito| 8.99|
 Steak Salad| 8.99|
 Steak Bowl| 8.99|
 Barbacoa Burrito| 8.99|
 Chicken Soft Tacos| 8.75|
 Chicken Salad Bowl| 8.75|
Chicken Crispy Tacos| 8.75|
 Veggie Salad| 8.49|
 Veggie Crispy Tacos| 8.49|
 Chicken Burrito| 8.49|
 Crispy Tacos| 7.4|
 Salad| 7.4|
 Bowl| 7.4|
 Burrito| 7.4|
 6 Pack Soft Drink| 6.49|
 Chips and Guacamole| 4.45|
 Nantucket Nectar| 3.39|
 Izze| 3.39|
Chips and Mild Fr...| 3.0|
Chips and Tomatil...| 2.95|
Chips and Tomatil...| 2.95|
Chips and Roasted...| 2.95|
Chips and Roasted...| 2.39|
Chips and Tomatil...| 2.39|
Chips and Fresh T...| 2.39|
Chips and Tomatil...| 2.39|
 Chips| 2.15|
 Side of Chips| 1.69|
 Canned Soft Drink| 1.25|
 Bottled Water| 1.09|
 Canned Soda| 1.09|
+--------------------+----------+

In [0]:
#What was the quantity of the most expensive item ordered?
df1.orderBy('item_price',ascending=False).select('*').show(1)

+--------+--------+--------------------+------------------+----------+------+
order_id|quantity| item_name|choice_description|item_price|Row_id|
+--------+--------+--------------------+------------------+----------+------+
 1443| 15|Chips and Fresh T...| NULL| 44.25| 3598|
+--------+--------+--------------------+------------------+----------+------+
only showing top 1 row

In [0]:
#How many times were a Veggie Salad Bowl ordered?
df1.filter(df1['item_name']=='Veggie Salad Bowl').show()

+--------+--------+-----------------+--------------------+----------+------+
order_id|quantity| item_name| choice_description|item_price|Row_id|
+--------+--------+-----------------+--------------------+----------+------+
 83| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 0|
 128| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 1|
 195| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 2|
 207| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 3|
 394| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 8.75| 4|
 536| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 8.75| 5|
 760| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 6|
 869| 1|Veggie Salad Bowl|[Tomatillo Red Ch...| 11.25| 7|
 896| 1|Veggie Salad Bowl|[Roasted Chili Co...| 8.75| 8|
 913| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 8.75| 9|
 1066| 1|Veggie Salad Bowl|[Roasted Chili Co...| 8.75| 10|
 1289| 1|Veggie Salad Bowl|[Tomatillo Red Ch...| 11.25| 11|
 1321| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 8.75| 12|
 1646| 1|Veggie Salad Bowl|[Tomatillo Red Ch...| 11.25| 13|
 1677| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 14|
 1700| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 11.25| 15|
 1805| 1|Veggie Salad Bowl|[Tomatillo Green ...| 8.75| 16|
 1818| 1|Veggie Salad Bowl|[Fresh Tomato Sal...| 8.75| 17|
+--------+--------+-----------------+--------------------+----------+------+

In [0]:
df1.filter(df1['item_name']=='Veggie Salad Bowl').count()

Out[72]: 18